### Understand the code step by step

In [1]:
import numpy as np

In [2]:
input_texts = ['hello', 'worl', 'great to see you', 'never ending']
input_characters = set()

In [3]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
print(max_encoder_seq_length)
min_encoder_seq_length = min([len(txt) for txt in input_texts])
print(min_encoder_seq_length)

16
4


In [4]:
for input_text in input_texts:
    for char in input_text:
            if char not in input_characters:
                input_characters.add(char)


input_characters = sorted(list(input_characters))
print(input_characters)
for i, char in enumerate(input_characters):
    print("i, char -> ", i, char)

[' ', 'a', 'd', 'e', 'g', 'h', 'i', 'l', 'n', 'o', 'r', 's', 't', 'u', 'v', 'w', 'y']
i, char ->  0  
i, char ->  1 a
i, char ->  2 d
i, char ->  3 e
i, char ->  4 g
i, char ->  5 h
i, char ->  6 i
i, char ->  7 l
i, char ->  8 n
i, char ->  9 o
i, char ->  10 r
i, char ->  11 s
i, char ->  12 t
i, char ->  13 u
i, char ->  14 v
i, char ->  15 w
i, char ->  16 y


In [5]:
input_token_index = dict([char, i] for i, char in enumerate(input_characters))

print("input_token_index -> ", input_token_index)

input_token_index ->  {' ': 0, 'a': 1, 'd': 2, 'e': 3, 'g': 4, 'h': 5, 'i': 6, 'l': 7, 'n': 8, 'o': 9, 'r': 10, 's': 11, 't': 12, 'u': 13, 'v': 14, 'w': 15, 'y': 16}


In [6]:
num_encoder_tokens = len(input_characters)

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

print("input texts length -> ", len(input_texts))
print("max encoder seq length -> ", max_encoder_seq_length)
print("num encoder tokens -> ", num_encoder_tokens)
print(encoder_input_data) # prints 4x16x17 array of zeros
#meaning -> 4 dimensions, 16 rows & 17 columns

input texts length ->  4
max encoder seq length ->  16
num encoder tokens ->  17
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
